In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import yfinance as yf
from datetime import datetime
import scipy.optimize as sco

In [ ]:
#1. LOAD DATA
# ==========================
symbols = ["TSLA", "BND", "SPY"]
start = "2015-07-01"
end = "2025-07-31"

data = {}
for sym in symbols:
    df = yf.download(sym, start=start, end=end, progress=False, auto_adjust=False)
    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    data[sym] = df

In [ ]:
#2. FORECAST TSLA with ARIMA
# ==========================
tsla = data["TSLA"]["Adj Close"]

# Train-test split (train: up to end of 2023, test: 2024-2025)
train = tsla.loc[:'2023-12-31']
test = tsla.loc['2024-01-01':]

model = auto_arima(train, seasonal=False, trace=False, stepwise=True)
forecast, conf_int = model.predict(n_periods=len(test), return_conf_int=True)


In [ ]:

# Plot forecast vs actual
plt.figure(figsize=(12,6))
plt.plot(train.index, train, label="Train")
plt.plot(test.index, test, label="Test")
plt.plot(test.index, forecast, label="Forecast")
plt.fill_between(test.index, conf_int[:,0], conf_int[:,1], color='pink', alpha=0.3)
plt.title("TSLA Price Forecast (ARIMA)")
plt.legend()
plt.show()

In [ ]:
# Use 12-month future forecast for portfolio expected return
future_forecast, _ = model.predict(n_periods=252, return_conf_int=True)  # ~252 trading days
tsla_expected_return = (future_forecast[-1] / tsla.iloc[-1] - 1)  # % change over 1 year


In [ ]:
# 3. HISTORICAL RETURNS FOR BND & SPY
# ==========================
returns = pd.DataFrame()
for sym in symbols:
    returns[sym] = data[sym]["Adj Close"].pct_change()

# Annualized historical returns for BND & SPY
bnd_expected_return = returns["BND"].mean() * 252
spy_expected_return = returns["SPY"].mean() * 252

In [ ]:
# Expected returns vector
exp_returns = np.array([tsla_expected_return, bnd_expected_return, spy_expected_return])

# Covariance matrix (annualized)
cov_matrix = returns.cov() * 252

In [ ]:
# 4. PORTFOLIO OPTIMIZATION (MPT)
# ==========================
def portfolio_performance(weights, returns, cov_matrix):
    port_return = np.dot(weights, returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = port_return / port_volatility
    return port_return, port_volatility, sharpe_ratio

def min_volatility(weights):
    return portfolio_performance(weights, exp_returns, cov_matrix)[1]

def neg_sharpe(weights):
    return -portfolio_performance(weights, exp_returns, cov_matrix)[2]

constraints = ({'type':'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = tuple((0, 1) for _ in range(3))
init_guess = [1/3, 1/3, 1/3]


In [ ]:
# Minimum Volatility Portfolio
min_vol = sco.minimize(min_volatility, init_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Maximum Sharpe Ratio Portfolio
max_sharpe = sco.minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=constraints)


In [ ]:
# 5. EFFICIENT FRONTIER
# ==========================
def efficient_frontier(returns, cov_matrix, num_points=100):
    results = np.zeros((3, num_points))
    weights_list = []
    for i in range(num_points):
        weights = np.random.random(3)
        weights /= np.sum(weights)
        weights_list.append(weights)
        ret, vol, sharpe = portfolio_performance(weights, returns, cov_matrix)
        results[0,i] = vol
        results[1,i] = ret
        results[2,i] = sharpe
    return results, weights_list

results, weights_list = efficient_frontier(exp_returns, cov_matrix)

plt.figure(figsize=(10,6))
plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='viridis', marker='o', s=10)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(portfolio_performance(min_vol.x, exp_returns, cov_matrix)[1],
            portfolio_performance(min_vol.x, exp_returns, cov_matrix)[0], 
            color='red', marker='*', s=200, label='Min Volatility')
plt.scatter(portfolio_performance(max_sharpe.x, exp_returns, cov_matrix)[1],
            portfolio_performance(max_sharpe.x, exp_returns, cov_matrix)[0], 
            color='blue', marker='*', s=200, label='Max Sharpe')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.legend()
plt.show()

In [ ]:
# 6. FINAL PORTFOLIO RECOMMENDATION
# ==========================
print("Expected Returns Vector (annualized):", exp_returns)
print("\nMinimum Volatility Portfolio Weights:", min_vol.x)
print("Performance (Return, Volatility, Sharpe):", portfolio_performance(min_vol.x, exp_returns, cov_matrix))

print("\nMaximum Sharpe Ratio Portfolio Weights:", max_sharpe.x)
print("Performance (Return, Volatility, Sharpe):", portfolio_performance(max_sharpe.x, exp_returns, cov_matrix))